# CADU endpoints demo

In this demo we will call the rs-server CADU HTTP endpoints:

  * List available CADU products
  * Download some products into local storage and S3 bucket
  * Monitor the download status from the database.

In [23]:
# Define some variables
endpoint="http://rs-server:8000/cadip/CADIP/cadu" # rs-server host = the container name
start="2014-01-01T12:00:00.000Z"
stop="2023-12-30T12:00:00.000Z"

In [24]:
# From a terminal, to list the available CADU products, we would use the curl command:
!set -x && curl -X GET "{endpoint}/list?start_date={start}&stop_date={stop}" -H "accept: application/json"

+ curl -X GET 'http://rs-server:8000/cadip/CADIP/cadu/list?start_date=2014-01-01T12:00:00.000Z&stop_date=2023-12-30T12:00:00.000Z' -H 'accept: application/json'
{"CADIP":[["2b17b57d-fff4-4645-b539-91f305c27c69","DCS_04_S1A_20231121072204051312_ch1_DSDB_00001.raw"],["2b17b57d-fff4-4645-b539-91f305c27c60","DCS_04_S1A_20231121072204051312_ch1_DSDB_00002.raw"],["2b17b57d-fff4-4645-b539-91f305c27c61","DCS_04_S1A_20231121072204051312_ch1_DSDB_00003.raw"],["2b17b57d-fff4-4645-b539-91f305c27c62","DCS_04_S1A_20231121072204051312_ch1_DSDB_00004.raw"],["2b17b57d-fff4-4645-b539-91f305c27c63","DCS_04_S1A_20231121072204051312_ch1_DSDB_00005.raw"],["2b17b57d-fff4-4645-b539-91f305c27c64","DCS_04_S1A_20231121072204051312_ch1_DSDB_00006.raw"],["2b17b57d-fff4-4645-b539-91f305c27c65","DCS_04_S1A_20231121072204051312_ch1_DSDB_00007.raw"],["some_id_2","DCS_04_S1A_20231121072204051312_ch1_DSDB_00060.raw"],["some_id_3","DCS_04_S1A_20231121072204051312_ch2_DSDB_00046.raw"],["some_id_4","DCS_04_S1A_202311210722

In [64]:
# But let's do it in python so it's easier to parse results
import requests
import pprint 

data = requests.get(f"{endpoint}/list", {"start_date": start, "stop_date": stop})
assert data.status_code == 200

# Get the returned products as (id,name) lists
products = data.json()["CADIP"]
assert len(products) == 10
pprint.PrettyPrinter(indent=4).pprint(products)

# Keep only the names
product_names = [name for id, name in products]

[   [   '2b17b57d-fff4-4645-b539-91f305c27c69',
        'DCS_04_S1A_20231121072204051312_ch1_DSDB_00001.raw'],
    [   '2b17b57d-fff4-4645-b539-91f305c27c60',
        'DCS_04_S1A_20231121072204051312_ch1_DSDB_00002.raw'],
    [   '2b17b57d-fff4-4645-b539-91f305c27c61',
        'DCS_04_S1A_20231121072204051312_ch1_DSDB_00003.raw'],
    [   '2b17b57d-fff4-4645-b539-91f305c27c62',
        'DCS_04_S1A_20231121072204051312_ch1_DSDB_00004.raw'],
    [   '2b17b57d-fff4-4645-b539-91f305c27c63',
        'DCS_04_S1A_20231121072204051312_ch1_DSDB_00005.raw'],
    [   '2b17b57d-fff4-4645-b539-91f305c27c64',
        'DCS_04_S1A_20231121072204051312_ch1_DSDB_00006.raw'],
    [   '2b17b57d-fff4-4645-b539-91f305c27c65',
        'DCS_04_S1A_20231121072204051312_ch1_DSDB_00007.raw'],
    ['some_id_2', 'DCS_04_S1A_20231121072204051312_ch1_DSDB_00060.raw'],
    ['some_id_3', 'DCS_04_S1A_20231121072204051312_ch2_DSDB_00046.raw'],
    ['some_id_4', 'DCS_04_S1A_20231121072204051312_ch2_DSDB_00060.raw']]


In [61]:
# Request database using the psycopg2 pluging for postgres
!pip install psycopg2-binary
import psycopg2
from os import environ as env

# Close old connection if any
try:
    db.close()
    db_connection.close()
except Exception:
    pass

# Open connection to the database
db_connection = psycopg2.connect(
    host=env.get("POSTGRES_HOST"),
    port=env.get("POSTGRES_PORT"),
    database=env.get("POSTGRES_DB"),
    user=env.get("POSTGRES_USER"),
    password=env.get("POSTGRES_PASSWORD"))    
db = db_connection.cursor()

In [63]:
# The "list" endpoint has initialised the database with the products info.
# Print the first 2 products info.
db.execute ("select * from cadu_download_status limit 2")
db.fetchall()

[(1,
  '2b17b57d-fff4-4645-b539-91f305c27c69',
  'DCS_04_S1A_20231121072204051312_ch1_DSDB_00001.raw',
  datetime.datetime(2023, 11, 26, 17, 1, 39, 528000),
  None,
  None,
  'NOT_STARTED',
  None),
 (2,
  '2b17b57d-fff4-4645-b539-91f305c27c60',
  'DCS_04_S1A_20231121072204051312_ch1_DSDB_00002.raw',
  datetime.datetime(2023, 11, 26, 17, 1, 39, 528000),
  None,
  None,
  'NOT_STARTED',
  None)]

In [68]:
s3_bucket = "s3://test-data"
local_download_dir = "/tmp/local"
name="DCS_04_S1A_20231121072204051312_ch1_DSDB_00001.raw"
requests.get(endpoint, {"name": name, "local": local_download_dir, "obs": s3_bucket})


ConnectionError: HTTPConnectionPool(host='rs-server', port=8000): Max retries exceeded with url: /cadip/CADIP/cadu?name=DCS_04_S1A_20231121072204051312_ch1_DSDB_00001.raw&local=%2Ftmp%2Flocal&obs=s3%3A%2F%2Ftest-data (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f7c2ebe3250>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [58]:
import asyncio

s3_bucket = "s3://test-data"
local_download_dir = "/tmp/local"

# Call the CADIP endpoint to download one product in background and upload it to the S3 bucket.
async def download_one(name):    

    data = requests.get(endpoint, {"name": name, "local": local_download_dir, "obs": s3_bucket})
    assert data.status_code == 200
    
  await asyncio.sleep(1)
  print(f"hello: {i}")

async def other():
    print ("end")

async def root():
    async with asyncio.TaskGroup() as tg:
        for i in range (5):
            tg.create_task(test(i))
            tg.create_task (other())

await (root())
        
# print ("end")

# async def main():
#     return await asyncio.gather(test(1), test(2))

# asyncio.run (main())
# print("end")

end
hello: 0
hello: 1
hello: 2
hello: 3
hello: 4


In [17]:
# Clean everything
db.close()
db_connection.close()